# Día 3 — DataFrames con Polars para Customer Experience (CX)

> **Lectura obligatoria:** Vink, R. (2022). *Polars: A DataFrame Library for Rust and Python*  
> **Dataset utilizado:** NPS_data.csv – encuesta pública de Net Promoter Score (link raw permanente)  

En este notebook practicarás Polars con **micro‑retos** centrados en **Net Promoter Score (NPS)**, la métrica estrella de CX.

Cada reto sigue el formato TDAH‑friendly:

| Paso | Contenido |
|------|-----------|
| 📋 **Tarea** | Qué debes hacer, explicado paso a paso |
| 🎯 **Por qué lo haces** | Impacto directo en CX / Data Science |
| 🧠 **Lógica** | Razonamiento o teoría breve |
| 🔧 **Ejercicio** | Celda de código con comentarios **TODO** |
| 🔍 **Solución comentada** | Respuesta correcta (comentada, no ejecuta) |
| ✅ **Validación** | Tests automáticos, muestran ✅ si todo OK |
| 💡 **Reflexión** | Cierre sobre el aprendizaje y su aplicación práctica |

> ⚙️ **Requisitos previos**  
> ```bash
> pip install polars==0.20.0
> ```


## 🔹 Reto 1 — Carga y exploración básica del NPS

| Paso | |
|-----|-----|
| 📋 **Tarea** | Cargar el CSV de NPS directo desde GitHub Raw y revisar su estructura. |
| 🎯 **Por qué lo haces** | Entender las columnas y tipos asegura limpieza previa a cualquier métrica CX. |
| 🧠 **Lógica** | Polars detecta tipos al vuelo; validar `dtypes` evita errores río abajo. |


In [1]:
!pip install polars==0.20.0

   ---------------------------------------- 0.0/24.7 MB ? eta -:--:--
   --- ------------------------------------ 2.1/24.7 MB 16.8 MB/s eta 0:00:02
   ---- ----------------------------------- 2.6/24.7 MB 6.6 MB/s eta 0:00:04
   ----- ---------------------------------- 3.1/24.7 MB 8.0 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/24.7 MB 5.7 MB/s eta 0:00:04
   -------- ------------------------------- 5.2/24.7 MB 5.3 MB/s eta 0:00:04
   ------------- -------------------------- 8.1/24.7 MB 6.7 MB/s eta 0:00:03
   ---------------- ----------------------- 10.2/24.7 MB 7.2 MB/s eta 0:00:02
   ------------------ --------------------- 11.5/24.7 MB 7.2 MB/s eta 0:00:02
   -------------------- ------------------- 12.6/24.7 MB 7.0 MB/s eta 0:00:02
   ---------------------- ----------------- 13.6/24.7 MB 7.1 MB/s eta 0:00:02
   ------------------------- -------------- 15.7/24.7 MB 7.2 MB/s eta 0:00:02
   ------------------------- -------------- 15.7/24.7 MB 7.2 MB/s eta 0:00:02

In [ ]:
# import sys 
# Instalación de polars (Opcional)
# !{sys.executable} -m pip install -U polars

Defaulting to user installation because normal site-packages is not writeable
  Using cached polars-1.31.0-cp39-abi3-win_amd64.whl.metadata (15 kB)
Using cached polars-1.31.0-cp39-abi3-win_amd64.whl (35.2 MB)


In [40]:
# 🔧 EJERCICIO
import polars as pl

# Carga el CSV del ejercicio a realizar
url = "https://raw.githubusercontent.com/abeltavares/nps_performance_analysis/main/NPS_data.csv" 
nps = pl.read_csv(url)
print(nps.head())
print(nps.dtypes)


shape: (5, 6)
┌────────────┬─────────────────────────────────┬─────────┬────────────┬───────────────┬──────────┐
│ year_month ┆ patient_uuid                    ┆ Region  ┆ Company    ┆ survey_result ┆ sessions │
│ ---        ┆ ---                             ┆ ---     ┆ ---        ┆ ---           ┆ ---      │
│ i64        ┆ str                             ┆ str     ┆ str        ┆ i64           ┆ i64      │
╞════════════╪═════════════════════════════════╪═════════╪════════════╪═══════════════╪══════════╡
│ 202104     ┆ bf728334-f122-4070-ad90-6e95d9… ┆ Lisboa  ┆ MedicoTech ┆ 10            ┆ 2        │
│ 202104     ┆ 2930f901-69ca-4191-ac1e-04e837… ┆ Porto   ┆ MedicoTech ┆ null          ┆ 2        │
│ 202104     ┆ 8ec034a1-50fc-4f70-ae69-0150f4… ┆ Coimbra ┆ VitaLife   ┆ null          ┆ 5        │
│ 202104     ┆ 6810ed87-0840-4e9d-b64d-b68ce9… ┆ Lisboa  ┆ VitaLife   ┆ 7             ┆ 35       │
│ 202104     ┆ 6194c31e-010c-45e8-b3a9-8213f3… ┆ Porto   ┆ HealthPlus ┆ 8             ┆ 35     

In [12]:
# ✅ VALIDACIÓN
assert 'survey_result' in nps.columns, "Falta la columna 'survey_result'"
assert nps.height > 0, "El DataFrame está vacío"
print("✅ ¡Buen trabajo! El DataFrame se cargó correctamente.")


✅ ¡Buen trabajo! El DataFrame se cargó correctamente.


💡 **Reflexión**  
¿Cuáles columnas son más relevantes para futuras segmentaciones de CX? Anota tus ideas para el proyecto maestro.

## 🔹 Reto 2 — Cálculo del NPS mensual

| Paso | |
|-----|-----|
| 📋 **Tarea** | Calcular el Net Promoter Score por mes (`year_month`). |
| 🎯 **Por qué lo haces** | El NPS mensual revela picos de insatisfacción para accionar mejoras rapidas. |
| 🧠 **Lógica** | NPS = (Promotores − Detractores) / Total × 100. Promotor >8, Detractor <7. |


In [ ]:
# Realizamos imputación de los valores nulos en la columna 'survey_result'
# con la mediana de los valores existentes.
nps = nps.with_columns(pl.col("survey_result").fill_null(pl.col("survey_result").median()))
nps.head()

year_month,patient_uuid,Region,Company,survey_result,sessions
i64,str,str,str,f64,i64
202104,"""bf728334-f122-4070-ad90-6e95d9…","""Lisboa""","""MedicoTech""",10.0,2
202104,"""2930f901-69ca-4191-ac1e-04e837…","""Porto""","""MedicoTech""",9.0,2
202104,"""8ec034a1-50fc-4f70-ae69-0150f4…","""Coimbra""","""VitaLife""",9.0,5
202104,"""6810ed87-0840-4e9d-b64d-b68ce9…","""Lisboa""","""VitaLife""",7.0,35
202104,"""6194c31e-010c-45e8-b3a9-8213f3…","""Porto""","""HealthPlus""",8.0,35


In [107]:

monthly_nps = (
    nps
    .group_by("year_month") # Agrupamos por mes
    .agg([
        pl.len().alias("total"), # Contamos el total de respuestas
        pl.col("survey_result").filter(pl.col("survey_result") >= 7).len().alias("promotores"), # Contamos los promotores (>= 7)
        pl.col("survey_result").filter(pl.col("survey_result") < 7).len().alias("detractores"), # Contamos los detractores (< 7)
        pl.col("survey_result").mean().alias("media").round(2), # Calculamos la media de las respuestas
    ]).sort("year_month", descending=False)
    .with_columns(
        ((pl.col("promotores") - pl.col("detractores"))*100/pl.col("total")).round(1).alias("nps"), # Calculamos el NPS en %
        (pl.col("promotores") / pl.col("total") * 100).alias("%_promotores").round(1), # Porcentaje de promotores
        (pl.col("detractores") / pl.col("total") * 100).alias("%_detractores").round(1) # Porcentaje de detractores
    )
)

print(monthly_nps)

shape: (9, 8)
┌────────────┬───────┬────────────┬─────────────┬───────┬──────┬──────────────┬───────────────┐
│ year_month ┆ total ┆ promotores ┆ detractores ┆ media ┆ nps  ┆ %_promotores ┆ %_detractores │
│ ---        ┆ ---   ┆ ---        ┆ ---         ┆ ---   ┆ ---  ┆ ---          ┆ ---           │
│ i64        ┆ u32   ┆ u32        ┆ u32         ┆ f64   ┆ f64  ┆ f64          ┆ f64           │
╞════════════╪═══════╪════════════╪═════════════╪═══════╪══════╪══════════════╪═══════════════╡
│ 202104     ┆ 599   ┆ 514        ┆ 85          ┆ 8.31  ┆ 71.6 ┆ 85.8         ┆ 14.2          │
│ 202105     ┆ 1219  ┆ 1011       ┆ 208         ┆ 8.16  ┆ 65.9 ┆ 82.9         ┆ 17.1          │
│ 202106     ┆ 896   ┆ 756        ┆ 140         ┆ 8.2   ┆ 68.8 ┆ 84.4         ┆ 15.6          │
│ 202107     ┆ 1026  ┆ 891        ┆ 135         ┆ 8.35  ┆ 73.7 ┆ 86.8         ┆ 13.2          │
│ 202108     ┆ 800   ┆ 694        ┆ 106         ┆ 8.37  ┆ 73.5 ┆ 86.8         ┆ 13.2          │
│ 202109     ┆ 467   ┆ 410

In [108]:
# ✅ VALIDACIÓN
# Verifica que la columna 'nps' esté en el DataFrame y que sus valores estén entre -100 y 100
assert 'nps' in monthly_nps.columns, "No se creó la métrica 'nps'"
assert monthly_nps['nps'].min() >= -100 and monthly_nps['nps'].max() <= 100, "Valores NPS fuera de rango"
print("✅ ¡Buen trabajo! NPS mensual calculado.")


✅ ¡Buen trabajo! NPS mensual calculado.


💡 **Reflexión**  
¿Observas algún mes con NPS < 0? ¿Qué hipótesis de negocio explicarían esa caída y cómo las validarías?

## 🔹 Reto 3 — Segmentar NPS por tipo de visita

| Paso | |
|-----|-----|
| 📋 **Tarea** | Determinar si el NPS de los pacientes difieren por `Region`. |
| 🎯 **Por qué lo haces** | Diferentes journeys producen expectativas distintas; segmentar revela oportunidades específicas. |
| 🧠 **Lógica** | Filtrar y agrupar por columna `Region`. |


In [116]:

# Calculamos el NPS segmentado por región
segment_nps = (
    nps
    .group_by("Region")
    .agg([
        pl.len().alias("total"),
        pl.col("survey_result").filter(pl.col("survey_result") >= 7).len().alias("promotores"),
        pl.col("survey_result").filter(pl.col("survey_result") < 7).len().alias("detractores")
    ])
    .with_columns(
        ((pl.col("promotores") - pl.col("detractores")) / pl.col("total") * 100).round(1).alias("nps")
    )
)

segment_nps


Region,total,promotores,detractores,nps
str,u32,u32,u32,f64
"""Coimbra""",1506,1304,202,73.2
"""Lisboa""",2978,2558,420,71.8
"""Porto""",1531,1305,226,70.5


La `Region` de Coimbra presenta el mejor NPS.

In [118]:
# ✅ VALIDACIÓN
assert 'nps' in segment_nps.columns, "No se calculó NPS por segmento"
assert len(segment_nps) >= 2, "Deben existir al menos dos tipos de visita"
print("✅ ¡Buen trabajo! NPS segmentado por tipo de visita.")


✅ ¡Buen trabajo! NPS segmentado por tipo de visita.


💡 **Reflexión**  
¿Qué acciones CX priorizarías para el segmento con peor NPS? Escribe dos ideas rápidas en tu cuaderno.